# 1.Poolingレイヤのbackwardの理解 

##  1-1. 実装コードを観察する
Poolingクラスの行数は多くないので、全体をのせる。

In [1]:
# coding: utf-8
import sys, os
sys.path.append(os.pardir)  # 親ディレクトリのファイルをインポートするための設定
import numpy as np
import matplotlib.pyplot as plt
from common.util import *

#from common/layers.py
class Pooling:
    def __init__(self, pool_h, pool_w, stride=2, pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad
        
        self.x = None
        self.arg_max = None

    def forward(self, x):
        N, C, H, W = x.shape
        out_h = int(1 + (H - self.pool_h) / self.stride)
        out_w = int(1 + (W - self.pool_w) / self.stride)

        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        col = col.reshape(-1, self.pool_h*self.pool_w)

        arg_max = np.argmax(col, axis=1)
        out = np.max(col, axis=1)
        out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)

        self.x = x
        self.arg_max = arg_max

        return out

    def backward(self, dout):
        dout = dout.transpose(0, 2, 3, 1)
        
        pool_size = self.pool_h * self.pool_w
        dmax = np.zeros((dout.size, pool_size))
        dmax[np.arange(self.arg_max.size), self.arg_max.flatten()] = dout.flatten()
        dmax = dmax.reshape(dout.shape + (pool_size,)) 
        
        dcol = dmax.reshape(dmax.shape[0] * dmax.shape[1] * dmax.shape[2], -1)
        dx = col2im(dcol, self.x.shape, self.pool_h, self.pool_w, self.stride, self.pad)
        
        return dx


## 1-2. backwardでは何をやっているのか？概要をざっくりと理解する

(im2colを一度理解してしまえば、）概念理解的にはそれほど難解なことをやっていないようである。
テキストのＰ２２８〜Ｐ２２９の解説にあるとおり、以下になる。
1. 次層からの逆伝搬の入力(dout)のサイズ×プールサイズ(pool_h*pool_w)の要素が0の行列を生成する
2. っっｘ
3. っっｘ

それでは、backwardの動作をサンプルを作りながらステップバイステップで確認していく。

## 1-3. 全体をサンプル作って試していく

In [2]:
from my_common.util import init_sample_matrix

### 複数画像枚、複数チャンネルの場合
単純のため、strideを2、padを0に設定する(ch07/simple_convnet.pyの実装を見ると、strideは2で、padは0である)

In [3]:
#入力画像の高さ、幅、および、チャネル数。画像の枚数
H  = 4 
W  = 5 
C  = 2 
N  = 2 #入力画像の枚数

#poolingレイヤの基本パラメータ
pool_h = 2
pool_w = 2
stride = 2
pad    = 0

#出力データ（画像）の高さと幅（自動的に計算される。）
out_h = int(1 + (H - pool_h) / stride)
out_w = int(1 + (W - pool_w) / stride)

print("出力画像の高さ=%d と 幅=%d" % (out_h ,out_w))

#xの用意
print("=== preparing of x===")
x11 = init_sample_matrix(filter_num = 1, channel=1, height=H, width=W) #filter番号(=画像番号)を識別する数値を与える(1,2~)
x12 = init_sample_matrix(filter_num = 1, channel=2, height=H, width=W)
x21 = init_sample_matrix(filter_num = 2, channel=1, height=H, width=W) #filter番号(=画像番号)を識別する数値を与える(1,2~)
x22 = init_sample_matrix(filter_num = 2, channel=2, height=H, width=W)
x = np.array([[x11,x12],[x21,x22]])
print(x.shape)
print(x)

出力画像の高さ=2 と 幅=2
=== preparing of x===
(2, 2, 4, 5)
[[[[1111 1112 1113 1114 1115]
   [1121 1122 1123 1124 1125]
   [1131 1132 1133 1134 1135]
   [1141 1142 1143 1144 1145]]

  [[1211 1212 1213 1214 1215]
   [1221 1222 1223 1224 1225]
   [1231 1232 1233 1234 1235]
   [1241 1242 1243 1244 1245]]]


 [[[2111 2112 2113 2114 2115]
   [2121 2122 2123 2124 2125]
   [2131 2132 2133 2134 2135]
   [2141 2142 2143 2144 2145]]

  [[2211 2212 2213 2214 2215]
   [2221 2222 2223 2224 2225]
   [2231 2232 2233 2234 2235]
   [2241 2242 2243 2244 2245]]]]


### backwardの処理をまずは実行してみる

In [4]:
#まず、Poolingクラスを生成してforwardさせ、backwardの準備をさせる
pool = Pooling(pool_h, pool_w)
out = pool.forward(x)
print(out)

[[[[1122. 1124.]
   [1142. 1144.]]

  [[1222. 1224.]
   [1242. 1244.]]]


 [[[2122. 2124.]
   [2142. 2144.]]

  [[2222. 2224.]
   [2242. 2244.]]]]


ここまでは、7.4.4_understanding_Pooling_forward_methodの"## 1-4. [参考]  複数画像枚、複数チャンネルの場合"の出力結果と同じになっている。ＯＫだね。pooling層の出力結果としてpoolのウインドウでフィルターした最大の要素が並んでいることもわかる。これをbackwardしたら一体どうなるのか？出力結果から先に見てみよう。

In [5]:
dout = pool.backward(out)
print(dout)

[[[[   0.    0.    0.    0.    0.]
   [   0. 1122.    0. 1124.    0.]
   [   0.    0.    0.    0.    0.]
   [   0. 1142.    0. 1144.    0.]]

  [[   0.    0.    0.    0.    0.]
   [   0. 1222.    0. 1224.    0.]
   [   0.    0.    0.    0.    0.]
   [   0. 1242.    0. 1244.    0.]]]


 [[[   0.    0.    0.    0.    0.]
   [   0. 2122.    0. 2124.    0.]
   [   0.    0.    0.    0.    0.]
   [   0. 2142.    0. 2144.    0.]]

  [[   0.    0.    0.    0.    0.]
   [   0. 2222.    0. 2224.    0.]
   [   0.    0.    0.    0.    0.]
   [   0. 2242.    0. 2244.    0.]]]]


なんと！forwardの出力結果を要素の１つとする、サイズがpool_h * pool_wの小行列がならび、サイズが入力xとあっている形になっていることがわかる。

### backwardの処理をステップ・バイ・ステップで実行してみる

In [8]:
#下準備
print("下準備")
dout = 0
pool = Pooling(pool_h, pool_w)
out = pool.forward(x)
dout = out
print("======= forwardの出力結果 =======")
print(dout)

下準備
[[[[1122. 1124.]
   [1142. 1144.]]

  [[1222. 1224.]
   [1242. 1244.]]]


 [[[2122. 2124.]
   [2142. 2144.]]

  [[2222. 2224.]
   [2242. 2244.]]]]
======= forwardの出力結果 =======
[[[[1122. 1124.]
   [1142. 1144.]]

  [[1222. 1224.]
   [1242. 1244.]]]


 [[[2122. 2124.]
   [2142. 2144.]]

  [[2222. 2224.]
   [2242. 2244.]]]]


In [12]:
print("======= transposeをかけるの出力結果 =======")
print("doutの軸を以下のように変換する")
print("　０軸目（画像インデックス...n）　　→０軸（そのまま）")
print("　１軸目（チャネルインデックス...h）→３軸") #hはchannelのh。列のcolumnとかぶるのでhを採用
print("　２軸目（行...r）　　　　　　　　　→１軸")
print("　３軸目（列...c）　　　　　　　　　→２軸")
print("・・・つまり、以下のように座標が変わる")
print("dout(n,h,r,c)　→　dout_t(n,r,c,h)")
print("dout(0,0,0,1)<要素1124> →　dout_t(0,0,1,0)<要素1124>")
print("dout(0,0,1,0)<要素1142> →　dout_t(0,1,0,0)<要素1142>")
dout = dout.transpose(0, 2, 3, 1)
print(dout)

======= transposeをかけるの出力結果 =======
doutの軸を以下のように変換する
　０軸目→０軸（そのまま）
　１軸目→３軸
　２軸目→１軸
　３軸目→２軸
・・・つまり、以下のように座標が変わる
dout(i,j,k,l)　→　dout_t(i,k,l,j)
dout(0,1,0,1)<要素1124> →　dout_t(0,0,1,1)<要素1124>
[[[[1122. 1222.]
   [1124. 1224.]]

  [[1142. 1242.]
   [1144. 1244.]]]


 [[[2122. 2222.]
   [2124. 2224.]]

  [[2142. 2242.]
   [2144. 2244.]]]]


画像毎に、チャネル毎に並んでいたものが、画像毎に列の方向にチャネルが並んでいることがわかる。

In [26]:
print("==== dmaxの初期化 ====")
pool_size = pool.pool_h * pool.pool_w
dmax = np.zeros((dout.size, pool_size))
print(dout.size)
print(pool_size)
print(dmax.shape)

==== dmaxの初期化 ====
16
4
(16, 4)


In [27]:
print("==== dmaxへの値(dout)の代入 ===")
print(dmax)
dmax[np.arange(pool.arg_max.size), pool.arg_max.flatten()] = dout.flatten()
print(np.arange(pool.arg_max.size)) #pool出力結果の全要素数
print(pool.arg_max.shape)
print(pool.arg_max)
print(pool.arg_max.flatten()) #テンソル対応？
print(dmax)
print("dmaxの各行(0~15)のforward時に最大値を示した要素のインデックス(列)に対して値を代入するのを一気にやっている")

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
(16,)
[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]
[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]
[[   0.    0.    0. 1122.]
 [   0.    0.    0. 1222.]
 [   0.    0.    0. 1124.]
 [   0.    0.    0. 1224.]
 [   0.    0.    0. 1142.]
 [   0.    0.    0. 1242.]
 [   0.    0.    0. 1144.]
 [   0.    0.    0. 1244.]
 [   0.    0.    0. 2122.]
 [   0.    0.    0. 2222.]
 [   0.    0.    0. 2124.]
 [   0.    0.    0. 2224.]
 [   0.    0.    0. 2142.]
 [   0.    0.    0. 2242.]
 [   0.    0.    0. 2144.]
 [   0.    0.    0. 2244.]]


In [28]:
print("dmaxの整形処理1・・・入力xのサイズに整形する処理のはじめのほう")
dmax = dmax.reshape(dout.shape + (pool_size,)) 
print(dout.shape)
print(pool_size)
print(dmax)

(2, 2, 2, 2)
4
[[[[[   0.    0.    0. 1122.]
    [   0.    0.    0. 1222.]]

   [[   0.    0.    0. 1124.]
    [   0.    0.    0. 1224.]]]


  [[[   0.    0.    0. 1142.]
    [   0.    0.    0. 1242.]]

   [[   0.    0.    0. 1144.]
    [   0.    0.    0. 1244.]]]]



 [[[[   0.    0.    0. 2122.]
    [   0.    0.    0. 2222.]]

   [[   0.    0.    0. 2124.]
    [   0.    0.    0. 2224.]]]


  [[[   0.    0.    0. 2142.]
    [   0.    0.    0. 2242.]]

   [[   0.    0.    0. 2144.]
    [   0.    0.    0. 2244.]]]]]


In [30]:
dcol = dmax.reshape(dmax.shape[0] * dmax.shape[1] * dmax.shape[2], -1)
print(dmax.shape)
print(dmax.shape[0])
print(dmax.shape[1])
print(dmax.shape[2])
print(dmax.shape[0] * dmax.shape[1] * dmax.shape[2])
print(dcol)

(2, 2, 2, 2, 4)
2
2
2
8
[[   0.    0.    0. 1122.    0.    0.    0. 1222.]
 [   0.    0.    0. 1124.    0.    0.    0. 1224.]
 [   0.    0.    0. 1142.    0.    0.    0. 1242.]
 [   0.    0.    0. 1144.    0.    0.    0. 1244.]
 [   0.    0.    0. 2122.    0.    0.    0. 2222.]
 [   0.    0.    0. 2124.    0.    0.    0. 2224.]
 [   0.    0.    0. 2142.    0.    0.    0. 2242.]
 [   0.    0.    0. 2144.    0.    0.    0. 2244.]]


In [31]:
dx = col2im(dcol, pool.x.shape, pool.pool_h, pool.pool_w, pool.stride, pool.pad)
print(dx)

[[[[   0.    0.    0.    0.    0.]
   [   0. 1122.    0. 1124.    0.]
   [   0.    0.    0.    0.    0.]
   [   0. 1142.    0. 1144.    0.]]

  [[   0.    0.    0.    0.    0.]
   [   0. 1222.    0. 1224.    0.]
   [   0.    0.    0.    0.    0.]
   [   0. 1242.    0. 1244.    0.]]]


 [[[   0.    0.    0.    0.    0.]
   [   0. 2122.    0. 2124.    0.]
   [   0.    0.    0.    0.    0.]
   [   0. 2142.    0. 2144.    0.]]

  [[   0.    0.    0.    0.    0.]
   [   0. 2222.    0. 2224.    0.]
   [   0.    0.    0.    0.    0.]
   [   0. 2242.    0. 2244.    0.]]]]


これでpoolingのbackwardの処理が見終わった。